In [1]:
import re
import os
import pandas as pd
import numpy as np
import traceback2
from datetime import datetime
import pymysql
import numpy as np

where_map = {}
# pairs regexp: "\[([a-zA-z0-9$\-+τ\.%]*)\]-", "-\[([a-zA-z0-9$\-+τ]*)\]",  or ( c in ['τ'] for c in s)

In [2]:
def pair1(line):
#     print(line)
    try:
        is_3_pairs = line.split("]-[")
        if len(is_3_pairs) > 2:
            return None
        s_index = line.index("[") + 1
        s = line[s_index: line.index("]", s_index)]
        if all(ord(c) < 128 for c in s):
            return s
        else:
            print(line)
            return None
    except:
        return ''

def pair2(line):
#     print(line)
    s_index = line.index("-[") + 2
    s = line[s_index: line.index("]", s_index)]
#     print(s)
    if all(ord(c) < 128 for c in s):
        return s
    else:
        print(line)
        return ''

In [3]:
def apr(line):
    apr = "Year [0-9\,\.]+\%"
    x = re.findall(apr, line)
    return x[0].replace("Year ", "").replace("%", "").strip() if len(x) else 0

In [4]:
def tvl(line):
    tvl = "TVL: \$[0-9\.\,]+"
    x = re.findall(tvl, line)
    return x[0].replace("TVL: $", "").replace("%", "").replace(",","").strip()

In [5]:
def where(line):
    if 'INFO' in line:
        return line.replace("INFO", "").replace('\n', '').replace(":", "").replace("//", "://").replace('*', '').strip()
    return None

In [6]:
def get_single_pair(line):
    exp = "\d* *.*Price:"
    x = re.findall(exp, line)
    if not len(x) or "TVL:" in line:
#         print(">>>>>>>>>>>",line)
        return
    pair = x[0].replace(" Price:", "")
    return pair[pair.rindex(" "):].strip() if " " in pair else pair

def get_stacked_tvl(line):
    exp = "\([0-9\,\.$]+\)"
    x = re.findall(exp, line)
    return x[0].replace("(", "").replace(")", "").replace("$", "").replace(",","").strip()


In [38]:
def is_end_of_block(line):
#     print(line, line == "" or len(re.findall("INFO[ ]+: ", line)))
    return line.strip() == "" or len(re.findall("INFO[ ]+: ", line)) or "******" in line

def read_block(lines, start_index):
    block = []
#     print("Block Start Line No.: ", start_index + 1)
    while start_index < len(lines):
        line = lines[start_index]
        start_index = start_index + 1
        if not is_end_of_block(line):
            block.append(line.strip())
        else:
            break
    return [block, start_index]

def read_blocks_from_files(file):
    f = open(file)
    lines = f.readlines()
    blocks = []
    block = []
    block_start_line_no = []
    line_no = 0
    start_index = 0
    while start_index < len(lines):
        line = lines[start_index]
        if not is_end_of_block(lines[start_index]):
            block_and_index = read_block(lines, start_index)
#             print(block_and_index[0])
            if block_and_index is not None:
                block_start_line_no.append(start_index + 1)
                start_index = block_and_index[1]
                blocks.append(block_and_index[0])
            else:
                start_index = start_index + 1
        elif len(re.findall("INFO[ ]+: ", line)):
            block_start_line_no.append(start_index + 1)
            blocks.append([line])
            start_index = start_index + 1
        else:
            start_index = start_index + 1
    return [blocks, block_start_line_no]

def read_blocks_from_files_as(file):
    f = open(file)
    lines = f.readlines()
    blocks = []
    block = []
    block_start_line_no = []
    line_no = 0
    for line in lines:
        line_no = line_no + 1
        line = line.strip()
        if line == "" or len(re.findall("INFO[ ]+: ", line)):
            blocks.append(block)
            block_start_line_no.append(line_no)
            if len(line):
                blocks.append([line])
                block_start_line_no.append(line_no + 1)
            block = []
        else:
            block.append(line)
            
    blocks.append(block)
    return [blocks, block_start_line_no]

def get_pair_info_from_block(block, wh, network):
    tvl_index = 0
    if network == 'BSC':
        tvl_index = 3
    if len(block) == 1:
        if 'INFO' in block[0]:
            return [where(block[0])]
        else:
            return [wh] if len(wh) else None
    if len(block) and pair1(block[0]) and not get_single_pair(block[0]):
        apr_index = -1
        for index in range(1, len(block)):
            if 'APR: ' in block[index]:
                apr_index = index
                break
        return [pair1(block[0]), pair2(block[0]), get_single_tvl(block[tvl_index]) if tvl_index > 0 else tvl(block[tvl_index]), apr(block[apr_index]) if apr_index > 1 else 0, where_map[wh] if wh in where_map else wh, network]
    elif len(block) > 2 and get_single_pair(block[0]):
#         print('singel')
        if network == 'BSC':
            tvl_index = 1
        apr_index = -1
        for index in range(1, len(block)):
            if 'APR: ' in block[index]:
                apr_index = index
                break
        return [get_single_pair(block[0]), "", get_single_tvl(block[1]), apr(block[apr_index]) if apr_index > 1 else 0, where_map[wh] if wh in where_map else wh, network]
    elif len(block) > 2 and get_alternate_pairs(block[0]):
        print(get_alternate_pairs(block[0]))
        tvl_index = 0
        for index in range(1, len(block)):
            if 'Staked: ' in block[index]:
                tvl_index = index
                break
        apr_index = -1
        for index in range(1, len(block)):
            if 'APR: ' in block[index]:
                apr_index = index
                break
        pairs = get_alternate_pairs(block[0])
        return [pairs[0], pairs[1], get_single_tvl(block[tvl_index]), apr(block[apr_index]), where_map[wh] if wh in where_map else wh, network]
    return None


def get_pairs(blocks, network, block_start_line_no):
    where = ""
    pairs = []
    index = -1
    for block in blocks:
#         print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        index = index + 1
        try:
            pair = get_pair_info_from_block(block, where, network)
            if pair == None:
                print('no Pair....\n',block)
                print("Block Start Line No.: ", block_start_line_no[index] if index < len(block_start_line_no) else -1)
#             if len(where) == 0 and pair and len(pair) > 2:
#                 print(block, network)
            if pair is not None and len(pair) > 0:
                if len(pair) == 1 and pair[0] and len(pair[0]):
                    where = pair[0]
                else:
                    pairs.append(pair)
        except Exception as e:
            print("Exception", traceback2.format_exc())
            print(block[0])
    return pairs
        

In [39]:
def get_alternate_pairs(line):
    spl = line.split("/")
    print(spl)
    if spl and len(spl) > 1:
        pair1 = spl[0].strip()
        print(pair1)
        pair2 = spl[1].strip()
        print(pair2)
        pair2 = pair2[0 : pair2.index(' ')]
        return [pair1, pair2]
    return None
    

In [35]:
data_dir = './'
def conver_files_to_one_df(data_dir):
    all_pairs = []
    network = "Test"
    blocks = read_blocks_from_files(f"blocks.txt")
    block_start_line_no = blocks[1]
    blocks = blocks[0]
    pairs = get_pairs(blocks, network, block_start_line_no)
    all_pairs.extend(pairs)
    return all_pairs

In [40]:
pairs = conver_files_to_one_df(data_dir)

['DAI ($1.00)']
no Pair....
 ['DAI ($1.00)', 'Supplied : 1,036,988,170.35 ($1,036,988,170.35) at 2.56% APR', 'Borrowed : 781,734,522.07 ($781,734,522.07) at 3.77% APR', 'Reserves : 255,253,648.67 ($255,253,648.67)', 'Farming APR Supply 1.22% Borrow 2.44%', 'Net APR Supply 3.78% Borrow -1.33%']
Block Start Line No.:  1
['WMATIC Pool:']
no Pair....
 ['WMATIC Pool:', '--Reward: 50C', '--APR: 20.32%', '--TVL: $1,444,230', '--Your deposit: $0.00']
Block Start Line No.:  36
0 - [🌾]-[WETH] SLP [+] [-] [<=>] Price: $301.90 TVL: $62,953.20
['0 - [🌾]-[WETH] SLP [+] [-] [<=>] Price: $301.90 TVL: $62,953.20']
no Pair....
 ['0 - [🌾]-[WETH] SLP [+] [-] [<=>] Price: $301.90 TVL: $62,953.20', '🌾 Price: $10.45', 'WETH Price: $2,081.40', 'Staked: 201.8257 SLP ($60,930.57)', '🌾 Per Week: 95206.61 ($995,137.20)', 'APR: Day 233.32% Week 1633.23% Year 84928.04%', 'You are staking 0.00 [🌾]-[WETH] SLP ($0.00), 0.00% of the pool.', 'Stake 0.00 [🌾]-[WETH] SLP', 'Unstake 0.00 [🌾]-[WETH] SLP', 'Staking or unstaki

In [10]:
pair_df = pd.DataFrame(pairs, columns=['pair1', 'pair2', 'tvl', 'apr', 'where', 'network'])
pair_df['apy'] = np.nan

In [41]:
pairs

[['UNI-V2', '', '0.00', 0, '', 'Test'],
 ['USDC', 'FISH', '827218.73', '333.32', '', 'Test'],
 ['USDC', 'QUICK', '8351689.14', 0, '', 'Test'],
 ['WETH', '', '114447.52', '11303.68', '', 'Test'],
 ['FARM', '', '17370414.42', '39.86', '', 'Test'],
 ['FARM', 'WETH', '4637017.67', '31.71', '', 'Test'],
 ['DPI', 'WETH', '1882047.36', '1.17', '', 'Test'],
 ['Badger', 'wBTC', '29201.95', '43.36', '', 'Test'],
 ['curve.fi', 'renBTC', '35158.02', '24.53', '', 'Test'],
 ['VSD', 'USDT', '0.00', 0, '', 'Test'],
 ['BTCx5-USDC-1Jun21-LP', '', '852291.43', '244.92', '', 'Test'],
 ['YT-aUSDC-29DEC2022', 'USDC', '2756775.92', '309.89', '', 'Test']]